In [1]:
import sys
sys.path.append('../../30_data_tools')

In [2]:
from helper import load_dotenv
from get_labelstudio_data import get_tasks, get_results_of_project

In [3]:
import re
import pandas as pd
import pickle
import json
import requests
from PIL import Image
from datetime import datetime
from tqdm import tqdm

In [4]:
config = load_dotenv()

In [5]:
generated_images = pd.concat([pd.read_pickle(f) for f in config['GENERIC_INFORMATION_DATA_DIR'].glob('./*.pkl')])

generated_images = generated_images.loc[
    generated_images.ssim < 0.95
]

In [6]:
tasks = get_tasks(2)
results = get_results_of_project(2)

In [7]:
def get_masks_by_task( task, generated_images ):
    basic_name_task = re.match(r'.+\/(.+)\..+?\..?', task['data_path']).groups()[0]
    for method in ['multiply','screen','soft_light']:
        basic_name_task = basic_name_task.replace(method,'$PLACEHOLDER$')
    
    res = generated_images.loc[
        generated_images.basic_name == basic_name_task
    ]
    
    if res.shape[0] == 0:
        return None
        
    return res.bbox.to_list()

In [8]:
def create_annotation( task, masks, results ):
    with Image.open( task['data_path'] ) as img:
        img_size = img.size
     
    headers = {'Authorization': f'Token { config["LABEL_STUDIO_TOKEN"] }'}
    json_data = {
        "completed_by": 1,
        "result": [
            {
                "original_width": img_size[0],
                "original_height": img_size[1],
                "image_rotation": 0,
                "value": {
                    "x": m[0] / img_size[0] * 100,
                    "y": m[1] / img_size[1] * 100,
                    "width": m[2] / img_size[0] * 100,
                    "height": m[3] / img_size[1] * 100,
                    "rotation": 0,
                    "rectanglelabels": [
                        "unchecked_moire"
                    ]
                },
                "from_name": "label",
                "to_name": "image",
                "type": "rectanglelabels",
                "origin": "manual"
            } for m in masks
        ],
        "was_cancelled": False,
        "ground_truth": False,
        "draft_created_at": datetime.now().isoformat(),
        "task": task["id"],
        "project": task["project_id"],
        "updated_by": 1,
        "parent_prediction": None,
        "parent_annotation": None
    }

    if len([r for r in results if r['img_name'] == task['data_path'].split('/')[-1]]) == 0:    
        requests.post(
            f'http://localhost:8080/api/tasks/{ task["id"] }/annotations/',
            headers=headers,
            json=json_data
        )

In [9]:
for t in tqdm(tasks):
    masks = get_masks_by_task( t, generated_images )

    if masks:
        create_annotation(
            t,
            masks,
            results
        )

 47%|████▋     | 413/879 [00:00<00:00, 553.81it/s]/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3182: DecompressionBombWarning: Image size (127715826 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 879/879 [00:11<00:00, 78.50it/s] 
